In [2]:
# HACK: use project root as the working directory 
from pathlib import Path

while Path.cwd().name != 'language-model-toxicity':
    %cd ..

/home/sam/Desktop/research/language-model-toxicity


In [3]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer

from models.affect_lm import AffectGPT2LMHeadModel
from utils.constants import OUTPUT_DIR

In [ ]:
model_dir = OUTPUT_DIR / 'affect' / 'affect_100k_1layer_randomsample_frozen' / 'finetune_output'
model = AffectGPT2LMHeadModel.from_pretrained(model_dir)

In [4]:
affect2vocab = model.affect.affect2vocab
weight = affect2vocab.weight
bias = affect2vocab.bias

In [25]:
values, indices = weight.sort(dim=0)

In [36]:
top_vocab = reversed(indices[:,0])[:100]

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [13]:
vocab = tokenizer.get_vocab()
vocab, _ = zip(*vocab.items())
vocab = pd.Series(vocab)
vocab

0                    !
1                    "
2                    #
3                    $
4                    %
             ...      
50252         Ġregress
50253        ĠCollider
50254      Ġinformants
50255           Ġgazed
50256    <|endoftext|>
Length: 50257, dtype: object

In [38]:
vocab[top_vocab.tolist()][:20]

31030         Ġbastard
12131            Ġ2020
30248            Ġlame
29078           volent
10225    Ġcorporations
12270            Ġdamn
28067            Ġgays
6945             ifies
22324           Ġidiot
18927        something
23778              hao
6806             arity
25164            Ġcrus
14372           ĠBaker
50112            crazy
17974            ieves
11456            antom
17626      Ġcigarettes
19132           Ġpiano
9828             Ġwarn
dtype: object

In [ ]:
import torch

from utils.generation import GPT2Generator
from utils.constants import OUTPUT_DIR, PERSPECTIVE_API_ATTRIBUTES_LOWER
from datasets.affect_dataset import create_affect_vector
from models.affect_lm import AffectGPT2LMHeadModel

model_dir = OUTPUT_DIR / 'affect' / 'affect_5k_1layer_randomsample_upsample_toxicity_5k' / 'finetune_output'
model = AffectGPT2LMHeadModel.from_pretrained(model_dir)
model.affect.beta = -1

generator = GPT2Generator(model)
device = generator.device

affects = [create_affect_vector(**{attribute: 1}) for attribute in PERSPECTIVE_API_ATTRIBUTES_LOWER]
affects.append(create_affect_vector())
affects = torch.tensor(affects).unsqueeze(dim=1).to(device)

# prompt = GPT2Generator.STOP_TOKEN
prompt = "Hello there"
out = generator([prompt] * len(affects), max_len=50, affect_labels=affects)

for attribute, gen in zip(PERSPECTIVE_API_ATTRIBUTES_LOWER + ('neutral',), out):
    print(f'Generation with {attribute}:'.upper(), prompt + gen, sep='\n', end='\n\n')

